<a href="https://colab.research.google.com/github/JhonUsuga/ModelosEntregaFinal/blob/main/05%20-%20modelo%20con%20preprocesado%20Optuna_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

udea_ai_4_eng_20251_pruebas_saber_pro_colombia_path = kagglehub.competition_download('udea-ai-4-eng-20251-pruebas-saber-pro-colombia')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/submission_example.csv
/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv
/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/test.csv


In [ ]:
!pip install optuna --quiet

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import optuna

In [ ]:
train = pd.read_csv("/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv")
test = pd.read_csv("/kaggle/input/udea-ai-4-eng-20251-pruebas-saber-pro-colombia/test.csv")

In [ ]:
cols_nulos = [
    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
    'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
    'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE'
]
for col in cols_nulos:
    modo = train[col].mode()[0]
    train[col] = train[col].fillna(modo)
    test[col] = test[col].fillna(modo)

# One-hot encoding
cols_cat = cols_nulos + ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_PRIVADO_LIBERTAD']
train = pd.get_dummies(train, columns=cols_cat, drop_first=True)
test = pd.get_dummies(test, columns=cols_cat, drop_first=True)
                      # Limpiar columnas
train.columns = train.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
test.columns = test.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

# Definir X, y y test
X = train.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = train["RENDIMIENTO_GLOBAL"].map({'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3})
test_ids = test["ID"]

# Alinear columnas
X, test = X.align(test, join='left', axis=1, fill_value=0)
X = X.loc[:, ~X.columns.duplicated()]
test = test.loc[:, ~test.columns.duplicated()]
# Escalar coeficientes
scaler = MinMaxScaler()
for col in ['coef_1', 'coef_2', 'coef_3', 'coef_4']:
    X[col] = scaler.fit_transform(X[[col]])
    test[col] = scaler.transform(test[[col]])

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
        'n_jobs': -1
    }

    model = LGBMClassifier(**params)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        acc = accuracy_score(y_val, preds)
        scores.append(acc)

    return np.mean(scores)

In [ ]:
# Buscar mejores parámetros
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Mejores parámetros encontrados:")
print(study.best_params)

[I 2025-06-17 18:12:52,038] A new study created in memory with name: no-name-93c61723-c675-4b92-bf35-fe9f13c6ab3e


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2357
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 730
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-06-17 18:16:52,390] Trial 0 finished with value: 0.43014440433212997 and parameters: {'n_estimators': 252, 'learning_rate': 0.06640502524558906, 'max_depth': 10, 'num_leaves': 78, 'min_child_samples': 29, 'subsample': 0.8991823890890407, 'colsample_bytree': 0.8371448759291527}. Best is trial 0 with value: 0.43014440433212997.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042293 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2081
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 592
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2087
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 594
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] St

[I 2025-06-17 18:20:50,012] Trial 1 finished with value: 0.41194801444043316 and parameters: {'n_estimators': 215, 'learning_rate': 0.01290344460238754, 'max_depth': 10, 'num_leaves': 74, 'min_child_samples': 55, 'subsample': 0.6012258435883708, 'colsample_bytree': 0.9572106433705473}. Best is trial 0 with value: 0.43014440433212997.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1895
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 499
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-06-17 19:50:19,619] Trial 28 finished with value: 0.430472202166065 and parameters: {'n_estimators': 256, 'learning_rate': 0.08463103038573341, 'max_depth': 10, 'num_leaves': 49, 'min_child_samples': 83, 'subsample': 0.6353561633039408, 'colsample_bytree': 0.6842364396227911}. Best is trial 23 with value: 0.431441155234657.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.040049 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1969
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 536
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-06-17 19:53:53,752] Trial 29 finished with value: 0.4307234657039711 and parameters: {'n_estimators': 231, 'learning_rate': 0.06960053133845955, 'max_depth': 12, 'num_leaves': 62, 'min_child_samples': 69, 'subsample': 0.5361549305856451, 'colsample_bytree': 0.5731470287695759}. Best is trial 23 with value: 0.431441155234657.


Mejores parámetros encontrados:
{'n_estimators': 234, 'learning_rate': 0.08785004728728527, 'max_depth': 12, 'num_leaves': 58, 'min_child_samples': 92, 'subsample': 0.7951771899294886, 'colsample_bytree': 0.6959602436445751}


In [ ]:
# Entrenar modelo final con mejores parámetros
final_model = LGBMClassifier(**study.best_params)
preds_proba = np.zeros((test.shape[0], 4))
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
val_scores = []

for train_idx, val_idx in kf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    final_model.fit(X_train, y_train)
    val_pred = final_model.predict(X_val)
    acc = accuracy_score(y_val, val_pred)
    val_scores.append(acc)

    preds_proba += final_model.predict_proba(test)

# Predicción final
final_preds = np.argmax(preds_proba / 5, axis=1)
map_inv = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
pred_labels = [map_inv[i] for i in final_preds]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047676 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1829
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 466
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [ ]:
submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": pred_labels
})
submission.to_csv("/kaggle/working/submission.csv", index=False)

print(f"Accuracy promedio en validación: {np.mean(val_scores):.5f}")
print("submission.csv generado")

Accuracy promedio en validación: 0.43117
submission.csv generado
